In [6]:
def read_ble_fingerprint(paths, beacon_list=None, window_size=1000):
    fp_features, fp_locs, all_beacon = [], [], set()
    
    for path in paths:
        with open(path, 'r') as f:
            print("in ble")
            print(path)
            lines = [x.strip() for x in f.readlines()]

        points = lines[1].split()
        points = [tuple(map(float, p.split(','))) for p in points]
        timestamps = list(map(int, lines[2].split()))

        idx = -1
        for line in lines[3:]:
            currT, *readings = line.split(' ')
            currT = int(currT)

            while currT >= timestamps[idx + 1]:
                idx += 1
                dx = points[idx + 1][0] - points[idx][0]
                dy = points[idx + 1][1] - points[idx][1]

            ratio = (currT - timestamps[idx]) / (timestamps[idx + 1] - timestamps[idx])
            x = points[idx][0] + ratio * dx
            y = points[idx][1] + ratio * dy
            # print(x,y)

            max_r = -100
            filtered_readings = []
            for r in readings:
                uuid, major, minor, reading, last_scan_time = r.split(',')
                if currT - int(last_scan_time) < window_size:
                    all_beacon.add(uuid.lower() + ',' + major + ',' + minor)
                    max_r = float(reading) if float(reading) > max_r else max_r
                    filtered_readings.append(r)

            if max_r > -100:
                fp_locs.append((x, y))
                fp_features.append(filtered_readings)

    all_beacon = list(sorted(all_beacon))
    if beacon_list is None:
        beacon_list = all_beacon

    for i, fp_feature in enumerate(fp_features):
        feature = [0 for _ in beacon_list]
        for r in fp_feature:
            uuid, major, minor, reading, _ = r.split(',')
            key = uuid.lower() + ',' + major + ',' + minor
            if key in beacon_list:
                feature[beacon_list.index(key)] = float(reading)
        fp_features[i] = feature

    return np.array(fp_features), fp_locs, all_beacon

In [7]:
import os
import sys
import glob
import numpy as np
sys.path.append(os.path.abspath(os.path.join('..')))

def read_wifi_fingerprint(paths):
    fp_features, fp_locs, ap_list = [], [], set()

    for path in paths:
        print(path)
        with open(path, 'r') as f:
            lines = [x.strip() for x in f.readlines()]

        points = lines[1].split()
        points = [tuple(map(float, p.split(','))) for p in points]
        timestamps = list(map(int, lines[2].split()))

        idx = -1
        for line in lines[3:]:
            currT, *readings = line.split(' ')
            currT = int(currT)

            while currT >= timestamps[idx + 1]:
                idx += 1
                dx = points[idx + 1][0] - points[idx][0]
                dy = points[idx + 1][1] - points[idx][1]

            ratio = (currT - timestamps[idx]) / (timestamps[idx + 1] - timestamps[idx])
            x = points[idx][0] + ratio * dx
            y = points[idx][1] + ratio * dy

            max_r = -100
            for r in readings:
                ap, reading = r.split(',')
                ap_list.add(ap.lower())
                max_r = float(reading) if float(reading) > max_r else max_r

            if max_r > -70:
                fp_locs.append((x, y))
                fp_features.append(readings)

    ap_list = list(sorted(ap_list))

    for i, fp_feature in enumerate(fp_features):
        feature = [0 for _ in ap_list]
        for r in fp_feature:
            ap, reading = r.split(',')
            if ap in ap_list:
                feature[ap_list.index(ap.lower())] = float(reading)
        fp_features[i] = feature

    return np.array(fp_features), fp_locs, ap_list
dataset_dir = 'data/HKUST_1F_Path2'
window_size = 3000
fp_features, fp_locs, beacon_list1 = read_ble_fingerprint(glob.glob(os.path.join(dataset_dir, 'n*_Ble.txt')), window_size=window_size)
#fp_features, fp_locs, ap_list1 = read_wifi_fingerprint(glob.glob(os.path.join(dataset_dir, 'n*_WiFi.txt')))
#print(fp_features)
#print(beacon_list1)

[]
[]
